# L1 and L2 Regularization
###### -----------------------------------------------------------------------------------------------------------------------------------------

### L1 weight regularization:

#### $L_{1w}$ = $\lambda \sum_{m}$ |$w_{m}$|
###### ------------------------------------------------------------------------------------------

### L1 bias regularization:

#### $L_{1b}$ = $\lambda \sum_{n}$ |$b_{n}$| 
###### ------------------------------------------------------------------------------------------

### L2 weight regularization:

#### $L_{2w}$ = $\lambda \sum_{m}$ $w_{m}^2$
###### ------------------------------------------------------------------------------------------

### L2 bias regularization:

#### $L_{2b}$ = $\lambda \sum_{n}$ $b_{n}^2$ 
###### ------------------------------------------------------------------------------------------

### Overall loss:

#### Loss = DataLoss + $L_{1w}$ + $L_{1b}$ + $L_{2w}$ + $L_{2b}$

## Updated Layer_Dense Class

In [1]:
import numpy as np


class Layer_Dense:
    def __init__(
        self,
        n_inputs,
        n_neurons,
        weight_regularizer_l1=0,
        weight_regularizer_l2=0,
        bias_regularizer_l1=0,
        bias_regularizer_l2=0,
    ):
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

        self.weight_regularizer_l1 = weight_regularizer_l1
        self.weight_regularizer_l2 = weight_regularizer_l2
        self.bias_regularizer_l1 = bias_regularizer_l1
        self.bias_regularizer_l2 = bias_regularizer_l2

    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)

        if self.weight_regularizer_l1 > 0:
            dL1 = np.ones_like(self.weights)
            dL1[self.weights < 0] = -1
            self.dweights += self.weight_regularizer_l1 * dL1

        if self.weight_regularizer_l2 > 0:
            self.dweights += 2 * self.weight_regularizer_l2 * self.weights

        if self.bias_regularizer_l1 > 0:
            dL1 = np.ones_like(self.biases)
            dL1[self.biases < 0] = -1
            self.dbiases += self.bias_regularizer_l1 * dL1

        if self.bias_regularizer_l2 > 0:
            self.dbiases += 2 * self.bias_regularizer_l2 * self.biases

        self.dinputs = np.dot(dvalues, self.weights.T)

## Updated Loss Class

In [2]:
class Loss:
    def regularization_loss(self, layer):
        regularization_loss = 0

        if layer.weight_regularizer_l1 > 0:
            regularization_loss += layer.weight_regularizer_l1 * np.sum(
                np.abs(layer.weights)
            )

        if layer.weight_regularizer_l2 > 0:
            regularization_loss += layer.weight_regularizer_l2 * np.sum(
                layer.weights * layer.weights
            )

        if layer.bias_regularizer_l1 > 0:
            regularization_loss += layer.bias_regularizer_l1 * np.sum(
                np.abs(layer.biases)
            )

        if layer.bias_regularizer_l2 > 0:
            regularization_loss += layer.bias_regularizer_l2 * np.sum(
                layer.biases * layer.biases
            )

        return regularization_loss

    def calculate(self, output, y):
        sample_losses = self.forward(output, y)
        data_loss = np.mean(sample_losses)
        return data_loss